In [1]:
%reset -f

In [2]:
# Close all widgets
import ipywidgets
ipywidgets.Widget.close_all()

# Widden display of notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

%load_ext autoreload
%autoreload 2
import gc # Garbage collected
import numpy as np
import sys, os
import time
import tifffile
import matplotlib.pyplot as plt
import scipy
from tkinter import Tk
from tkinter.filedialog import askopenfilename, askopenfilenames
import torch

In [3]:
print(sys.path)
from scripts import *

['/Documents/tools/anaconda3/envs/als/lib/python36.zip', '/Documents/tools/anaconda3/envs/als/lib/python3.6', '/Documents/tools/anaconda3/envs/als/lib/python3.6/lib-dynload', '', '/home/eboigne/.local/lib/python3.6/site-packages', '/Documents/tools/anaconda3/envs/als/lib/python3.6/site-packages', '/Documents/tools/anaconda3/envs/als/lib/python3.6/site-packages/IPython/extensions', '/home/eboigne/.ipython', '/home/eboigne/.local/share/JetBrains/Toolbox/apps/PyCharm-P/ch-0/212.5284.44/plugins/python/helpers/pydev', '/home/eboigne/.local/share/JetBrains/Toolbox/apps/PyCharm-P/ch-0/212.5284.44/plugins/python/helpers-pro/jupyter_debug']


In [5]:
torch.cuda.empty_cache()
gc.collect()

0

In [6]:
assert torch._C._cuda_getDeviceCount() > 0, 'No GPU detected'
print(torch.cuda.current_device())

0


In [7]:
path_save = '/Documents/nersc_als/2022_wood/'

In [8]:
use_dark_from_scan = False
if not use_dark_from_scan:
    h5py_file_dark = read_h5_file(path_save)[0]

Using file: /Documents/nersc_als/2022_wood/20220501_094201_21_dark_2560x1396.h5
	- 10 dark fields
	- 0 white fields
	- 385 projections
	- Image size: (1396, 2560)


In [9]:
use_flat_from_scan = True
if not use_flat_from_scan:
    h5py_file_flat = read_h5_file(path_save)[0]

In [10]:
h5py_file = read_h5_file(path_save)[0]

Using file: /Documents/nersc_als/2022_wood/20220501_172426_24_SampleB2_2xLens_01_pre_2625.h5
	- 10 dark fields
	- 100 white fields
	- 5249 projections
	- Image size: (1396, 2560)


In [19]:
# Acquisition parameters
pixel_size_microns = 3.24 # [microns]
N_angles_per_half_circle = 1313
N_half_circles = 2 # With half-circles, and 0-180 range: the 180 angle is repeated (acquired twice)

# Pre-processing parameters
bin_factor = 2 # Bin projection, flat & dark images by this factor
bin_factor_angle = 1 # Bin projection images along the angle direction by this factor
bin_gpu_chunk_size = 10 * bin_factor * bin_factor + 3 # Max number of slices simultaneously processed on the GPU
filter_type = 'ram-lak' # FBP filter. Note from ram-lak, apply Gaussian blur with radius sigma = 1 yields results really close to Parzen.
skip_first_flats = 20 # Skip the first few N flats, usually not as good quality.

# Double normalization parameters:
# normalize out fluctuations in photon counts between successive projection images
# using mean value in a window at the left and right of the FOV (where no sample is)
boolean_use_DN = True # Whether to use the double normalization
window_width_DN = 25 // bin_factor # Width of the window [in pixels]
window_cutEdge_DN = 10 // bin_factor # How far the window is from the left and right edges of the FOV [in pixels]
ind_range_DN = np.concatenate((np.arange(window_cutEdge_DN, window_width_DN,1),np.arange(h5py_file.width//bin_factor-(window_width_DN),2560//bin_factor-window_cutEdge_DN,1)))

# Outlier correction
use_outlier_correction = True
gpu_median_filter_chunk_size = 10 * bin_factor * bin_factor + 3 # Avoid divider of Nangles and Nflats. Even smaller because median filter requires large memory use. Check nvidia-smi
outlier_kernel_half_width = 2 # Make it at least bin_factor, otherwise can miss zinglers on first/last slices
outlier_zinger_threshold = 0.3

# Saving parameters:
proj_save_every = 31
save_sinogram = False
sino_save_every = 1

# For fast post-processing
crop_image = True
ind_image_crop_bottom = 690
ind_image_crop_top = 710

# Not recommended unless necessary
boolean_use_TN = False # True

In [ ]:
height = h5py_file.height

# Reconstruction
if N_half_circles == 2:
# For N_half_circles = 2, use ind_last_angle = 1 as last angle is the same as the first angle, and should thus be discarded
    ind_last_angle = 1 # Counting backwards. 0: using all angles, 1: Skip last angle, ...
else:
# For N_half_circles = 1, unsure since acquisition scheme changed in 2021.
    ind_last_angle = 0

N_avg = 1
angles_all = np.linspace(0, 2*np.pi, 2*(N_angles_per_half_circle*N_avg-(N_avg-1))-1)
angles_to_use = angles_all
angles_to_use1 = angles_to_use[:N_angles_per_half_circle-1]
if ind_last_angle > 0:
    angles_to_use2 = angles_to_use[N_angles_per_half_circle-1:-ind_last_angle]
else:
    angles_to_use2 = angles_to_use[N_angles_per_half_circle-1:]

pixel_size_cm = pixel_size_microns / 1.0e4 * bin_factor

ind_save_PMDOF_DN = np.concatenate((np.arange(0, (N_angles_per_half_circle-ind_last_angle)//bin_factor_angle, proj_save_every), np.arange((N_angles_per_half_circle-ind_last_angle)//bin_factor_angle-4,(N_angles_per_half_circle-ind_last_angle)//bin_factor_angle,1))).astype('int')
ind_save_sinogram = np.arange(0, h5py_file.height, sino_save_every).astype('int')

# ======================================== #

print2('\tImporting dark -- ',end='')
if use_dark_from_scan:
    dark = np.array(h5py_file['exchange']['data_dark']).astype('float32')
else:
    dark = np.array(h5py_file_dark['exchange']['data']).astype('float32')
print2('Done')

if crop_image:
    dark = dark[:,ind_image_crop_bottom:ind_image_crop_top,:]

if bin_factor>1:
    print2('\tBinning dark data ',end='')
    dark = fast_pytorch_bin_chunk(dark,bin_factor, chunk_size = bin_gpu_chunk_size)
    print2(' Done')
dark_avg = np.mean(dark,axis = 0)

ind_save_sinogram = ind_save_sinogram[ind_save_sinogram < dark.shape[1]]

tic = time.time()
print2('Processing file: '+str(h5py_file.path_file))

print2('\tImporting flat -- ',end='')
if use_flat_from_scan:
    flat = h5py_file['exchange']['data_white']
else:
    flat = h5py_file_flat['exchange']['data']
print2('Done')

flat = flat[skip_first_flats:]
if crop_image:
    flat = flat[:,ind_image_crop_bottom:ind_image_crop_top,:]

print2('\tImporting projections -- ',end='')
proj = h5py_file['exchange']['data']
print2('Done')

if crop_image:
    proj = proj[:,ind_image_crop_bottom:ind_image_crop_top,:]
bin_str = str(bin_factor_angle)+'x'+str(bin_factor)+'x'+str(bin_factor)

if bin_factor>1:
    print2('\tBinning data ',end='')
    flat = fast_pytorch_bin_chunk(flat,bin_factor, chunk_size = bin_gpu_chunk_size)
    if N_half_circles == 1:
        if ind_last_angle > 0:
            proj = proj[:-ind_last_angle]
        proj = fast_pytorch_bin_chunk(proj,bin_factor, bin_factor_angle = bin_factor_angle, chunk_size = bin_gpu_chunk_size)
    elif N_half_circles == 2:
        if ind_last_angle > 0:
            proj1 = fast_pytorch_bin_chunk(proj[:N_angles_per_half_circle-1],bin_factor, bin_factor_angle = bin_factor_angle, chunk_size = bin_gpu_chunk_size)
            proj2 = fast_pytorch_bin_chunk(proj[N_angles_per_half_circle-1:-ind_last_angle],bin_factor, bin_factor_angle = bin_factor_angle, chunk_size = bin_gpu_chunk_size)
        else:
            proj1 = fast_pytorch_bin_chunk(proj[:N_angles_per_half_circle-1],bin_factor, bin_factor_angle = bin_factor_angle, chunk_size = bin_gpu_chunk_size)
            proj2 = fast_pytorch_bin_chunk(proj[N_angles_per_half_circle-1:],bin_factor, bin_factor_angle = bin_factor_angle, chunk_size = bin_gpu_chunk_size)
    print2(' Done')
    ind_save_PMDOF_DN = ind_save_PMDOF_DN[ind_save_PMDOF_DN<proj2.shape[0]]
else:
    if N_half_circles == 2:
        if ind_last_angle > 0:
            proj1 = proj[:N_angles_per_half_circle-1]
            proj2 = proj[N_angles_per_half_circle-1:-ind_last_angle]
        else:
            proj1 = proj[:N_angles_per_half_circle-1]
            proj2 = proj[N_angles_per_half_circle-1:]

if use_outlier_correction:
    print2('\tRemoving outliers (zingers) for flats ',end='')
    flat = fast_pytorch_remove_zingers_chunk(flat, outlier_kernel_half_width, outlier_zinger_threshold, chunk_size = gpu_median_filter_chunk_size).astype('float32')
    print2(' Done')

    print2('\tRemoving outliers (zingers) for proj ',end='')
    if N_half_circles == 1:
        proj = fast_pytorch_remove_zingers_chunk(proj, outlier_kernel_half_width, outlier_zinger_threshold, chunk_size = gpu_median_filter_chunk_size).astype('float32')
    elif N_half_circles == 2:
        proj1 = fast_pytorch_remove_zingers_chunk(proj1, outlier_kernel_half_width, outlier_zinger_threshold, chunk_size = gpu_median_filter_chunk_size).astype('float32')
        proj2 = fast_pytorch_remove_zingers_chunk(proj2, outlier_kernel_half_width, outlier_zinger_threshold, chunk_size = gpu_median_filter_chunk_size).astype('float32')
    print2(' Done')

print2('\tComputing transmission -- ',end='')
flat += -dark_avg
flat_avg = np.mean(flat,axis = 0).astype('float32')
File.File(h5py_file.path_folder+h5py_file.file_name_noExtension+'/'+h5py_file.file_name_noExtension+'_bin'+bin_str+'_flat_avg', clear = True).saveTiff(flat_avg)
if N_half_circles == 1:
    proj += -dark_avg
    proj /= flat_avg
elif N_half_circles == 2:
    proj1 += -dark_avg
    proj2 += -dark_avg
    N_flat = flat.shape[0]
    proj1 /= flat_avg
    proj2 /= flat_avg
print2('Done')

if boolean_use_DN:
    print2('\tComputing double normalization -- ',end='')
    if N_half_circles == 1:
        double_norm = np.reshape(np.mean(proj[:,:,ind_range_DN], axis=(1,2)), [proj.shape[0], 1,1]) # One coeff per angle
        proj /= double_norm

        if boolean_use_TN:
            triple_norm = np.reshape(np.mean(proj[:,:,ind_range_DN], axis=(0,2)), [1, proj.shape[1],1]) # One coeff per slice (y-axis)
            triple_normB = np.copy(triple_norm)
            triple_normB[triple_normB<0.985] = 0.985
            triple_normB[triple_normB>1.015] = 1.015
            proj /= triple_normB

    elif N_half_circles == 2:
        double_norm1 = np.reshape(np.mean(proj1[:,:,ind_range_DN], axis=(1,2)), [proj1.shape[0], 1,1])
        double_norm2 = np.reshape(np.mean(proj2[:,:,ind_range_DN], axis=(1,2)), [proj2.shape[0], 1,1])

        proj1 /= double_norm1
        proj2 /= double_norm2

        if boolean_use_TN:
            triple_norm1 = np.reshape(np.mean(proj1[:,:,ind_range_DN], axis=(0,2)), [1, proj1.shape[1],1])
            triple_norm2 = np.reshape(np.mean(proj2[:,:,ind_range_DN], axis=(0,2)), [1, proj2.shape[1],1])

            triple_norm1B = np.copy(triple_norm1)
            triple_norm1B[triple_norm1B<0.985] = 0.985
            triple_norm1B[triple_norm1B>1.015] = 1.015

            triple_norm2B = np.copy(triple_norm2)
            triple_norm2B[triple_norm2B<0.985] = 0.985
            triple_norm2B[triple_norm2B>1.015] = 1.015

            proj1 /= triple_norm1B
            proj2 /= triple_norm2B
    print2('Done')
else:
    print2('\tSkipping double normalization')

print2('\tWriting PMDOF_DN to tif -- ',end='')
if N_half_circles == 1:
    suffix = '_bin'+bin_str+'_PMDOF_DN/'
    File.File(h5py_file.path_folder+h5py_file.file_name_noExtension+'/'+h5py_file.file_name_noExtension+suffix, clear = True).saveTiffStack(proj[ind_save_PMDOF_DN,:,:],ind=ind_save_PMDOF_DN)

elif N_half_circles == 2:
    suffix = '_bin'+bin_str+'_PMDOF_DN_a_0-180/'
    File.File(h5py_file.path_folder+h5py_file.file_name_noExtension+'/'+h5py_file.file_name_noExtension+suffix, clear = True).saveTiffStack(proj1[ind_save_PMDOF_DN,:,:],ind=ind_save_PMDOF_DN)

    suffix = '_bin'+bin_str+'_PMDOF_DN_b_180-360/'
    File.File(h5py_file.path_folder+h5py_file.file_name_noExtension+'/'+h5py_file.file_name_noExtension+suffix, clear = True).saveTiffStack(proj2[ind_save_PMDOF_DN,:,:],ind=ind_save_PMDOF_DN)
print2('Done')

print2('\tMaking sinograms -- ',end='')
if N_half_circles == 1:
    sino = -np.log(np.transpose(proj[:,ind_save_sinogram,:], [1, 0, 2]))
elif N_half_circles == 2:
    sino1 = -np.log(np.transpose(proj1[:,ind_save_sinogram,:], [1, 0, 2]))
    sino2 = -np.log(np.transpose(proj2[:,ind_save_sinogram,:], [1, 0, 2]))
    sino = np.concatenate((sino1,sino2),axis = 1)
print2('Done')

del proj, proj1, proj2
gc.collect()

if save_sinogram:
    print2('\tWriting sinograms to tif -- ',end='')
    if N_half_circles == 1:
        suffix = '_bin'+bin_str+'_sino/'
        File.File(h5py_file.path_folder+h5py_file.file_name_noExtension+'/'+h5py_file.file_name_noExtension+suffix, clear = True).saveTiffStack(sino,ind=ind_save_sinogram)
    elif N_half_circles == 2:
        suffix = '_bin'+bin_str+'_sino_a_0-180/'
        File.File(h5py_file.path_folder+h5py_file.file_name_noExtension+'/'+h5py_file.file_name_noExtension+suffix, clear = True).saveTiffStack(sino1,ind=ind_save_sinogram)
        suffix = '_bin'+bin_str+'_sino_b_180-360_flipped/'
        File.File(h5py_file.path_folder+h5py_file.file_name_noExtension+'/'+h5py_file.file_name_noExtension+suffix, clear = True).saveTiffStack(sino2,ind=ind_save_sinogram)
    print2('Done')

	Importing dark -- Done
	Binning dark data .......... Done
Processing file: /Documents/nersc_als/2022_wood/20220501_172426_24_SampleB2_2xLens_01_pre_2625.h5
	Importing flat -- Done
	Importing projections -- 

In [ ]:
ind_sino = 150
COR_table = np.linspace(-10, 0, 11)

if N_half_circles == 1:
    sino3 = sino
else:
    sino3 = sino1
angles = np.linspace(0, np.pi, sino3.shape[1], False)

rec = []
for COR in COR_table:
    rec.append(wrapper_ASTRA.FBP(sino3[ind_sino]/pixel_size_cm, filter_type=filter_type, angles = angles, center_rot = COR))
rec = np.array(rec)
suffix = '_tuningCOR'
File.File(h5py_file.path_folder+h5py_file.file_name_noExtension+'/'+h5py_file.file_name_noExtension+suffix, clear = True).saveTiffStack(rec)

In [ ]:
# Tune within ~0.1 pixels (then optimal is local anyway)
best_ind = 2

best_ind += -1 # Minus 1 from Fiji/ImageJ to Python
print('Slice #'+str(best_ind-1)+' - COR is '+str(COR_table[best_ind-1]))
print('Slice #'+str(best_ind)+' - COR is '+str(COR_table[best_ind]))
print('Slice #'+str(best_ind+1)+' - COR is '+str(COR_table[best_ind+1]))

In [ ]:
COR = -4.5 * bin_factor / bin_factor # Center-of-rotation [Pixel]
skip_every_slice = 10 # Only save every N-th slice

tic = time.time()
if N_half_circles == 1:
    print2('\tDoing FBP reconstruction -- ',end='')
    rec = []
    angles = np.linspace(0, np.pi, sino.shape[1], False)
    for this_sino in sino:
        rec.append(wrapper_ASTRA.FBP(this_sino/pixel_size_cm, filter_type=filter_type, angles = angles, center_rot = COR))
    rec = np.array(rec)
    print2('Done')

    print2('\tSaving FBP reconstruction ('+str(rec.shape[0])+' slices) -- ',end='')
    suffix = '_bin'+bin_str+'_FBP_COR_'+str(COR).zfill(2)+'/'
    File.File(h5py_file.path_folder+h5py_file.file_name_noExtension+'/'+h5py_file.file_name_noExtension+suffix, clear = True).saveTiffStack(rec,ind=ind_save_sinogram)
    print2('Done')

elif N_half_circles == 2:
    print2('\tDoing FBP reconstruction (1/2) -- ',end='')
    rec1 = []
    angles = angles_to_use1
    for this_sino in sino1[::skip_every_slice]:
        rec1.append(wrapper_ASTRA.FBP(this_sino/pixel_size_cm, filter_type=filter_type, angles = angles, center_rot = COR))
    rec1 = np.array(rec1)
    print2('Done (1/2)')

    print2('\tSaving FBP reconstruction (1/2) -- ',end='')
    suffix = '_bin'+bin_str+'_FBP_COR_'+str(COR).zfill(2)+'/'
    File.File(h5py_file.path_folder+h5py_file.file_name_noExtension+'/'+h5py_file.file_name_noExtension+suffix, clear = True).saveTiffStack(rec1,ind=ind_save_sinogram)
    print2('Done (1/2)')

    print2('\tDoing FBP reconstruction (2/2) -- ',end='')
    rec2 = []
    angles = angles_to_use2
    for this_sino in sino2[::skip_every_slice]:
        rec2.append(wrapper_ASTRA.FBP(this_sino/pixel_size_cm, filter_type=filter_type, angles = angles, center_rot = COR))
    rec2 = np.array(rec2)
    print2('Done (2/2)')

    print2('\tSaving FBP reconstruction (2/2) -- ',end='')
    suffix = '_bin'+bin_str+'_FBP_COR_'+str(COR).zfill(2)+'_b_180-360_flipped/'
    File.File(h5py_file.path_folder+h5py_file.file_name_noExtension+'/'+h5py_file.file_name_noExtension+suffix, clear = True).saveTiffStack(rec2,ind=ind_save_sinogram)
    print2('Done (2/2)')

    print2('\tSaving averaged of two half circles -- ',end='')
    suffix = '_bin'+bin_str+'_FBP_COR_'+str(COR).zfill(2)+'_c_averaged_half_circles/'
    File.File(h5py_file.path_folder+h5py_file.file_name_noExtension+'/'+h5py_file.file_name_noExtension+suffix, clear = True).saveTiffStack(0.5*(rec1+rec2),ind=ind_save_sinogram)
    print2('Done')

print2('This took '+str(time.time()-tic)+' s')
